### This script just adds the image flip function to the test and development sets.

Note that it won't run without the other code in `train_network.ipynb`. So I haven't tested it...

In [1]:
from torchvision.transforms.functional import hflip

In [ ]:
torchvision.transforms.functional.hflip(img)

## It's just the import statement and these lines in both the test and dev loops:

        # augment data with hflip
        imgs = torch.cat((imgs, imgs.hflip), 0)
        labels = torch.cat((labels, labels), 0)

In [ ]:
for epoch in range(max_epochs):
    tic = time.time()
    
    # train:
    model.train()
    running_loss = 0.
    for imgs, labels, _, _, _ in train_generator:
        imgs, labels = imgs.to(device), labels.to(device)
        
        # augment data with hflip
        imgs = torch.cat((imgs, imgs.hflip), 0)
        labels = torch.cat((labels, labels), 0)
        
        scores = model(imgs)
        loss = loss_fcn(scores,labels)
        
        running_loss += loss*labels.shape[0]
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    running_loss /= N_train
    print('E-%d Mean Training Loss = %.2f'%(epoch+1,running_loss))
    
    # development:
    model.eval()
    correct_predictions = 0
    for imgs, labels, _, _, _ in dev_generator:
        imgs, labels = imgs.to(device), labels.to(device)
        
        # augment data with hflip
        imgs = torch.cat((imgs, imgs.hflip), 0)
        labels = torch.cat((labels, labels), 0)
        
        scores = model(imgs)
        pred_class = scores.argmax(1)
        correct_predictions += (pred_class == labels).sum().item()
        
    dev_accuracy = 100 * correct_predictions/N_dev
    
    toc = time.time()
    print('E-%d Development Set Accuracy = %.2f%% in %.1f seconds'%(epoch+1,dev_accuracy,toc-tic))
    
    if dev_accuracy > best_accuracy:
        best_model = model
        best_accuracy = dev_accuracy